<a href="https://colab.research.google.com/github/Jihyun13579/ESAA/blob/main/0915(%EA%B8%88)%EC%84%B8%EC%85%98_%EB%AA%A8%EB%8D%B8%ED%9B%88%EB%A0%A8_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **| 모델 훈련 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch04 연습문제 1, 5, 9, 10
- 개념 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

### **1. 수백만 개의 특성을 가진 훈련 세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?**
___

답 : 수백만 개의 특성을 가지고 있는 데이터셋이라면 정규방정식이나 SVD보다는 SGD(but, 편향의 가능성), 미니배치 경사 하강법이 계산 속도에 용이할 것이다.

### **2. 배치 경사 하강법을 사용하고 에포크마다 검증 오차를 그래프로 나타내봤습니다. 검증 오차가 일정하게 상승되고 있다면 어떤 일이 일어나고 있는 걸까요? 이 문제를 어떻게 해결할 수 있나요?**
___

답: 검증 오차가 일정하게 상승하고 있다면, 경우에 따라서는 과적합이나, 일정하게 상승한다면 학습률이 너무 커 최적의 하이퍼 파라미터를 지나친 채 더 좋지 않은 파라미터로 발산하고 있는 상황일 수도 있다. 그렇기에, 훈련 세트의 에러 또한 상승하는지 확인하고, 훈련 세트의 에러 또한 증가하고 있다면 학습률을 낮추고 같은 현상이 발생하는지 확인해보는 과정이 필요하다.

### **3. 릿지 회귀를 사용했을 때 훈련 오차가 검증 오차가 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 $\alpha$를 증가시켜야 할까요 아니면 줄여야 할까요?**
___
검증 오차가 높지만 훈련 오차가 작다면 과적합을 의심해야 하지만, 훈련 오차와 검증 오차가 모두 비슷하게 높다면 모델이 제대로 훈련되지 않은 과소 적합을 의심해야 할 것이다. 편향이 큰 모델은 훈련 데이터에서 과소적합되기 쉬운 특성을 가졌으므로, 이 모델은 높은 편향이 문제일 것이고, 그렇기에 규제 하이퍼파라미터 $\alpha$를 감소시켜야 할 것이다.

### **4. 다음과 같이 사용해야 하는 이유는?**
___
- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀
- 릿지 회귀 대신 라쏘 회귀
- 라쏘 회귀 대신 엘라스틱넷

___
답:
- 규제가 있는 모델이 일반적으로 규제가 없는 모델보다 과적합을 방지하여 성능이 좋기 때문이다.
- 라쏘 회귀는 가중치를 완전히 0으로 만드는 경향이 있기 때문에, 변수를 줄여 feature selection을 해주는 모델을 생성하므로 일반적으로 라쏘 회귀가 더 좋을 것이다.
- 라쏘 회귀는 특성들이 서로 강하게 연관되어 있을 때 제대로 작동하지 않을 가능성이 있기에, 그런 안정성의 면에서 엘라스틱넷이 더 유용하다.

### **추가) 조기 종료를 사용한 배치 경사 하강법으로 소프트맥스 회귀를 구현해보세요(사이킷런은 사용하지 마세요)**


---

* 저자 답안 참고


In [15]:
import numpy as np

np.random.seed(2042)

1. 데이터 로드하기

In [16]:
from sklearn import datasets
iris = datasets.load_iris()
list(iris.keys())

['data',
 'target',
 'frame',
 'target_names',
 'DESCR',
 'feature_names',
 'filename',
 'data_module']

In [17]:
X = iris["data"][:, (2, 3)]  # 꽃잎 길이, 꽃잎 넓이
y = iris["target"]

In [18]:
# 모든 샘플에 편향 추가
X_with_bias = np.c_[np.ones([len(X), 1]), X]

2. 데이터 preprocessing

2-1. 데이터 나누는 과정 (train_test_split() 함수 없이)



In [20]:
test_ratio = 0.2
validation_ratio = 0.2
total_size = len(X_with_bias)

test_size = int(total_size * test_ratio)
validation_size = int(total_size * validation_ratio)
train_size = total_size - test_size - validation_size

rnd_indices = np.random.permutation(total_size)

X_train = X_with_bias[rnd_indices[:train_size]]
y_train = y[rnd_indices[:train_size]]
X_valid = X_with_bias[rnd_indices[train_size:-test_size]]
y_valid = y[rnd_indices[train_size:-test_size]]
X_test = X_with_bias[rnd_indices[-test_size:]]
y_test = y[rnd_indices[-test_size:]]

2-2. 원핫인코딩 함수

In [21]:
def to_one_hot(y):
    n_classes = y.max() + 1
    m = len(y)
    Y_one_hot = np.zeros((m, n_classes))
    Y_one_hot[np.arange(m), y] = 1
    return Y_one_hot

In [22]:
# 작동 확인
to_one_hot(y_train[:10])

array([[0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

2-3. 훈련 데이터셋, 테스트 데이터셋 타깃 클래스 확률 담은 행렬 생성하기

In [23]:
Y_train_one_hot = to_one_hot(y_train)
Y_valid_one_hot = to_one_hot(y_valid)
Y_test_one_hot = to_one_hot(y_test)

3. 소프트맥스 함수 생성하기

In [24]:
def softmax(logits):
    exps = np.exp(logits)
    exp_sums = np.sum(exps, axis=1, keepdims=True)
    return exps / exp_sums

4. 입력과 출력의 개수 정의하기

In [25]:
n_inputs = X_train.shape[1] # == 3 (특성 2개와 편향)
n_outputs = len(np.unique(y_train))   # == 3 (3개의 붓꽃 클래스)

5. 비용함수 구현하여 소프트맥스 모델 훈련시키기

In [26]:
eta = 0.01
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    if iteration % 500 == 0:
        loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
        print(iteration, loss)
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error)
    Theta = Theta - eta * gradients

0 2.0098317606794267
500 0.7775485994012685
1000 0.6358595299644286
1500 0.553854152323856
2000 0.5002301566083223
2500 0.46167353758270924
3000 0.43206307086901297
3500 0.4082683127711592
4000 0.3885249426733036
4500 0.3717545031055311
5000 0.35725297209557083


5-2. 훈련시킨 모델 파라미터 확인하기

In [27]:
Theta

array([[ 3.30012252, -0.19360882, -3.26721327],
       [-1.20747709, -0.58488635, -0.4152438 ],
       [-2.1970059 ,  0.25677352,  1.72547638]])

5-3. 검증 데이터셋에 대한 예측 정확도 확인하기

In [28]:
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

0.8666666666666667

6. L2 규제 추가한 모델 생성하기

In [29]:
eta = 0.1
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7
alpha = 0.1  # 규제 하이퍼파라미터

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    if iteration % 500 == 0:
        xentropy_loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
        l2_loss = 1/2 * np.sum(np.square(Theta[1:]))
        loss = xentropy_loss + alpha * l2_loss
        print(iteration, loss)
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error) + np.r_[np.zeros([1, n_outputs]), alpha * Theta[1:]]
    Theta = Theta - eta * gradients

0 2.968736899582155
500 0.5197864820932234
1000 0.4910558310637946
1500 0.48254537680629156
2000 0.4793184131301125
2500 0.47798748905462335
3000 0.4774151434812393
3500 0.4771630781432382
4000 0.4770504425998878
4500 0.47699964640820014
5000 0.47697660148778775


In [30]:
# 정확도 측정하기
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

0.9333333333333333

7. 함수에 조기 종료 추가해보기

In [33]:
eta = 0.1
n_iterations = 200001
m = len(X_train)
epsilon = 1e-7
alpha = 0.1  # 규제 하이퍼파라미터
best_loss = np.infty

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error) + np.r_[np.zeros([1, n_outputs]), alpha * Theta[1:]]
    Theta = Theta - eta * gradients

    logits = X_valid.dot(Theta)
    Y_proba = softmax(logits)
    xentropy_loss = -np.mean(np.sum(Y_valid_one_hot * np.log(Y_proba + epsilon), axis=1))
    l2_loss = 1/2 * np.sum(np.square(Theta[1:]))
    loss = xentropy_loss + alpha * l2_loss
    if iteration % 500 == 0:
        print(iteration, loss)
    if loss < best_loss:
        best_loss = loss
    else:
        print(iteration - 1, best_loss)
        print(iteration, loss, "조기 종료!")
        break

0 2.458575990910144
500 0.6254423728798993
1000 0.5882142845109087
1500 0.5763441453349325
2000 0.5715138720647914
2500 0.5693183708682336
3000 0.5682415558806976
3500 0.5676796835958234
4000 0.5673703526241188
4500 0.5671919301254713
5000 0.5670848797599896
5500 0.5670185618550828
6000 0.5669764386370089
6500 0.5669491753075212
7000 0.5669312853598879
7500 0.5669194300709608
8000 0.566911519229655
8500 0.5669062149620635
9000 0.5669026465764967
9500 0.5669002405051871
10000 0.566898615622627
10500 0.5668975171312067
11000 0.5668967739664758
11500 0.5668962709448538
12000 0.5668959303540688
12500 0.5668956996914155
13000 0.5668955434528538
13500 0.5668954376142012
14000 0.5668953659122526
14500 0.5668953173343964
15000 0.5668952844219793
15500 0.5668952621227049
16000 0.5668952470139689
16500 0.5668952367770348
17000 0.5668952298409461
17500 0.5668952251413407
18000 0.5668952219570736
18500 0.5668952197995353
19000 0.5668952183376672
19500 0.5668952173471589
20000 0.5668952166760258
20

In [34]:
# 정확도 측정해보기
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

0.9333333333333333

8. 테스스셋에 대한 모델 최종 정확도 예측하기

In [35]:
logits = X_test.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_test)
accuracy_score
# 결과 : 매우 높음!

1.0